In [18]:
%pip install geopandas shapely scikit-learn scipy numpy

  Using cached geopandas-1.1.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached shapely-2.1.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.5/32.5 MB 74.1 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 75.2 MB/s  0:00:00
   ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [pyproj]  WARNING: The script pyproj is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [geopandas]/4 [geopandas]
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
from shapely import wkt
from sklearn.cluster import KMeans, DBSCAN
from scipy.spatial import Voronoi, distance_matrix
import numpy as np
from collections import defaultdict

In [20]:
parks = pd.read_csv("assets/parks.csv")
rest = pd.read_csv("assets/restaurants.csv")
metro = pd.read_csv("assets/metro.csv",)

print(f"The number of [parks]: {len(parks):>8}")
print(f"The number of [restaurants]: {len(rest):>8}")
print(f"The number of [metros]: {len(metro):>8}")

The number of [parks]:     2055
The number of [restaurants]:   285290
The number of [metros]:      496


In [21]:
print(f"\n --- PARKS --- \n")
display(parks.head(n=5))

print(f"\n --- RESTAURANTS --- \n")
display(rest.head(n=5))

print(f"\n --- METRO --- \n")
display(metro.head(n=5))


 --- PARKS --- 



,ACQUISITIONDATE,ACRES,ADDRESS,BOROUGH,CLASS,COMMUNITYBOARD,COUNCILDISTRICT,DEPARTMENT,EAPPLY,GISOBJID,...,PIP_RATABLE,PRECINCT,RETIRED,SIGNNAME,SUBCATEGORY,TYPECATEGORY,US_CONGRESS,WATERFRONT,ZIPCODE,multipolygon
0,1972 Sep 22 12:00:00 AM,83.3,1000 RICHMOND TERRACE,R,PARK,501,49,R-01,Snug Harbor Cultural Center,"100,003,741",...,False,120,False,Snug Harbor Cultural Center,Large Park,Historic House Park,11,True,"10301, 10310",MULTIPOLYGON (((-74.10086903940999 40.64468646...
1,NaN,0.356,200-15 MCLAUGHLIN AV,Q,PARK,408,23,Q-08,McLaughlin Playground,"100,000,044",...,True,107,False,McLaughlin Playground,Neighborhood Plgd,Playground,5,False,11423,MULTIPOLYGON (((-73.76596231448613 40.72551186...
2,1926 Apr 29 12:00:00 AM,286.557,298 SATERLEE STREET,R,PARK,503,51,R-03,Conference House Park,"100,004,319",...,False,123,False,Conference House Park,Large Park,Nature Area,11,True,"10307, 10309",MULTIPOLYGON (((-74.25204073763335 40.50292259...
3,1990 Oct 26 12:00:00 AM,6.611,2 KENT STREET,B,PARK,301,33,B-01,WNYC Transmitter Park,"100,003,886",...,True,94,False,WNYC Transmitter Park,Neighborhood Park,Neighborhood Park,NaN,True,11222,MULTIPOLYGON (((-73.96140516032709 40.73032004...
4,1948 Jan 29 12:00:00 AM,1.534,255 PARKINSON AVENUE,R,PARK,502,50,R-02B,Old Town Playground,"100,004,659",...,True,122,False,Old Town Playground,JOP,Playground,11,False,10305,MULTIPOLYGON (((-74.08082911468377 40.59514394...



 --- RESTAURANTS --- 



,CAMIS,INSPECTION DATE,VIOLATION CODE
0,40375108,11/26/2025,04H
1,40375108,11/26/2025,10G
2,40395865,11/26/2025,08A
3,40395865,11/26/2025,10H
4,40395865,11/26/2025,04M



 --- METRO --- 



,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,CBD,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,False,N W,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0,0,0,NaN,POINT (-73.912034 40.775036)
1,R03,2,2,BMT,Astoria,Astoria Blvd,Q,False,N W,Elevated,40.770258,-73.917843,Astoria,Manhattan,1,1,1,NaN,POINT (-73.917843 40.770258)
2,R04,3,3,BMT,Astoria,30 Av,Q,False,N W,Elevated,40.766779,-73.921479,Astoria,Manhattan,0,0,0,NaN,POINT (-73.921479 40.766779)
3,R05,4,4,BMT,Astoria,Broadway,Q,False,N W,Elevated,40.761820,-73.925508,Astoria,Manhattan,0,0,0,NaN,POINT (-73.925508 40.76182)
4,R06,5,5,BMT,Astoria,36 Av,Q,False,N W,Elevated,40.756804,-73.929575,Astoria,Manhattan,0,0,0,NaN,POINT (-73.929575 40.756804)
